EDA — Credit Card Clients (Default / Late Payment Risk) by Kasia Mc art

In [ ]:
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go

from pathlib import Path

pd.set_option("display.max_columns", None)
pd.set_option("display.width", 200)


In [ ]:
# -----------------------------------------------------------------------------
# CONFIG
# -----------------------------------------------------------------------------
DATA_PATH = Path("default of credit card clients.xls")  # keep file next to this notebook, or update path
SHEET_NAME = 0

RANDOM_STATE = 42

print("Data path:", DATA_PATH.resolve() if DATA_PATH.exists() else DATA_PATH)


In [ ]:
# -----------------------------------------------------------------------------
# HELPERS (same style as stock EDA notebook)
# -----------------------------------------------------------------------------
def print_df(df: pd.DataFrame, name: str, n: int = 10):
    print("\n" + "-" * 95)
    print(f"[DATAFRAME] {name} | shape = {df.shape[0]:,} rows × {df.shape[1]:,} cols")
    print("-" * 95)
    print(df.head(n).to_string())

def section(title: str):
    print("\n" + "=" * 95)
    print(title)
    print("=" * 95)

def pct(x):
    return 100 * x


In [ ]:
# =============================================================================
# LOAD DATA
# =============================================================================
section("Loading data")

# NOTE: this dataset is often in legacy .xls format; pandas may require xlrd.
# If you get an error like "Missing optional dependency 'xlrd'", run:
#   pip install xlrd==2.0.1

df_raw = pd.read_excel(DATA_PATH, sheet_name=SHEET_NAME, header=1, engine="xlrd")

print_df(df_raw, "raw")
df_raw.info()


In [ ]:
# =============================================================================
# CLEAN COLUMN NAMES
# =============================================================================
section("Cleaning column names")

df = df_raw.copy()

# Standardize target name
df = df.rename(columns={"default payment next month": "DEFAULT_NEXT_MONTH"})

# Ensure ID exists
if "ID" in df.columns:
    df["ID"] = df["ID"].astype(int)

print_df(df, "cleaned")
print("Target distribution (counts):")
print(df["DEFAULT_NEXT_MONTH"].value_counts(dropna=False).to_string())
print("\nTarget distribution (rate %):")
print((df["DEFAULT_NEXT_MONTH"].value_counts(normalize=True) * 100).round(2).to_string())


## Data dictionary (high level)

**Target**
- `DEFAULT_NEXT_MONTH` — 1 = default next month, 0 = not default

**Static customer profile**
- `LIMIT_BAL` — credit limit
- `SEX`, `EDUCATION`, `MARRIAGE`, `AGE`

**Repayment status (behaviour)**
- `PAY_0`, `PAY_2`, `PAY_3`, `PAY_4`, `PAY_5`, `PAY_6`  
  (repayment status for the last 6 months; higher values = more delayed)

**Billing history**
- `BILL_AMT1` … `BILL_AMT6` — monthly statement balances

**Payment history**
- `PAY_AMT1` … `PAY_AMT6` — monthly payments made


In [ ]:
# =============================================================================
# DATA QUALITY CHECKS
# =============================================================================
section("Data quality checks")

print("Missing values per column:")
na = df.isna().sum().sort_values(ascending=False)
print(na[na>0].to_string() if (na>0).any() else "No missing values found.")

# Duplicate ID check (if ID column exists)
if "ID" in df.columns:
    dup = df["ID"].duplicated().sum()
    print(f"Duplicate IDs: {dup}")

# Basic describe
section("describe() — numeric")
print(df.describe().T.to_string())


In [ ]:
# =============================================================================
# CATEGORICAL VARIABLES — DISTRIBUTIONS
# =============================================================================
section("Categorical variables distributions")

cat_cols = ["SEX", "EDUCATION", "MARRIAGE"]
for c in cat_cols:
    print("\n" + "-"*80)
    print(f"{c} value_counts:")
    print(df[c].value_counts(dropna=False).to_string())
    fig = px.bar(
        df[c].value_counts().reset_index(),
        x="index", y=c,
        title=f"{c} distribution",
        labels={"index": c, c: "count"}
    )
    fig.show()


In [ ]:
# =============================================================================
# NUMERIC VARIABLES — DISTRIBUTIONS
# =============================================================================
section("Numeric variables distributions")

num_cols = ["LIMIT_BAL", "AGE"]
for c in num_cols:
    fig = px.histogram(df, x=c, nbins=50, title=f"{c} — histogram")
    fig.show()

# Boxplots for quick outlier scan
fig = px.box(df, y="LIMIT_BAL", title="LIMIT_BAL — boxplot")
fig.show()
fig = px.box(df, y="AGE", title="AGE — boxplot")
fig.show()


In [ ]:
# =============================================================================
# TARGET vs FEATURES (quick comparisons)
# =============================================================================
section("Target rate by categories")

for c in ["SEX", "EDUCATION", "MARRIAGE"]:
    g = (df.groupby(c)["DEFAULT_NEXT_MONTH"]
           .agg(["count","mean"])
           .rename(columns={"mean":"default_rate"}))
    g["default_rate_pct"] = (g["default_rate"]*100).round(2)
    print("\n" + "-"*80)
    print(f"DEFAULT rate by {c}:")
    print(g.sort_values("default_rate", ascending=False).to_string())

    fig = px.bar(g.reset_index(), x=c, y="default_rate_pct",
                 title=f"Default rate (%) by {c}",
                 labels={"default_rate_pct":"default rate (%)"})
    fig.show()


In [ ]:
# =============================================================================
# REPAYMENT STATUS (PAY_*) — KEY RISK SIGNALS
# =============================================================================
section("Repayment status (PAY_*) overview")

pay_cols = ["PAY_0","PAY_2","PAY_3","PAY_4","PAY_5","PAY_6"]
for c in pay_cols:
    print("\n" + "-"*80)
    print(f"{c} value_counts (top):")
    print(df[c].value_counts().head(12).to_string())

# Default rate by each PAY_* status (example: PAY_0)
section("Default rate by PAY_0")
g = (df.groupby("PAY_0")["DEFAULT_NEXT_MONTH"]
       .agg(["count","mean"])
       .rename(columns={"mean":"default_rate"}))
g["default_rate_pct"] = (g["default_rate"]*100).round(2)
print(g.sort_values("default_rate", ascending=False).to_string())

fig = px.bar(g.reset_index(), x="PAY_0", y="default_rate_pct",
             title="Default rate (%) by PAY_0",
             labels={"default_rate_pct":"default rate (%)"})
fig.show()


In [ ]:
# =============================================================================
# BILLING + PAYMENTS — MONTHLY PATTERNS (mean / median)
# =============================================================================
section("Billing and payments — monthly patterns")

bill_cols = [f"BILL_AMT{i}" for i in range(1,7)]
pay_amt_cols = [f"PAY_AMT{i}" for i in range(1,7)]

bill_stats = df[bill_cols].agg(["mean","median"]).T
pay_stats  = df[pay_amt_cols].agg(["mean","median"]).T

print("\nBilling stats (mean/median):")
print(bill_stats.to_string())
print("\nPayment stats (mean/median):")
print(pay_stats.to_string())

# Line charts
fig = go.Figure()
fig.add_trace(go.Scatter(x=bill_stats.index, y=bill_stats["mean"], mode="lines+markers", name="Bill mean"))
fig.add_trace(go.Scatter(x=bill_stats.index, y=bill_stats["median"], mode="lines+markers", name="Bill median"))
fig.update_layout(title="Billing amounts over last 6 months (mean vs median)", xaxis_title="month column", yaxis_title="amount")
fig.show()

fig = go.Figure()
fig.add_trace(go.Scatter(x=pay_stats.index, y=pay_stats["mean"], mode="lines+markers", name="Pay mean"))
fig.add_trace(go.Scatter(x=pay_stats.index, y=pay_stats["median"], mode="lines+markers", name="Pay median"))
fig.update_layout(title="Payment amounts over last 6 months (mean vs median)", xaxis_title="month column", yaxis_title="amount")
fig.show()


In [ ]:
# =============================================================================
# CORRELATIONS (numeric)
# =============================================================================
section("Correlation heatmap (numeric)")

# Correlation can be dominated by outliers; consider also Spearman in addition to Pearson.
num_for_corr = df.drop(columns=["ID"], errors="ignore").select_dtypes(include=[np.number]).copy()

corr_pearson = num_for_corr.corr(method="pearson")
corr_spearman = num_for_corr.corr(method="spearman")

fig = px.imshow(corr_pearson, title="Correlation heatmap (Pearson)", aspect="auto")
fig.show()

fig = px.imshow(corr_spearman, title="Correlation heatmap (Spearman)", aspect="auto")
fig.show()

# Quick top correlations with target
section("Top absolute correlations with target (Pearson)")
target_corr = corr_pearson["DEFAULT_NEXT_MONTH"].drop("DEFAULT_NEXT_MONTH").abs().sort_values(ascending=False)
print(target_corr.head(20).to_string())


In [ ]:
# =============================================================================
# FEATURE ENGINEERING (for ML)
# =============================================================================
section("Feature engineering")

df_fe = df.copy()

# --- Clean up category codes (common practice for this dataset) ---
# EDUCATION: 0, 5, 6 are often "unknown" -> group as 4 (Other)
df_fe["EDUCATION"] = df_fe["EDUCATION"].replace({0: 4, 5: 4, 6: 4})
# MARRIAGE: 0 often unknown -> group as 3 (Other)
df_fe["MARRIAGE"] = df_fe["MARRIAGE"].replace({0: 3})

# --- Payment ratio features ---
bill_cols = [f"BILL_AMT{i}" for i in range(1,7)]
pay_cols_amt = [f"PAY_AMT{i}" for i in range(1,7)]

for i in range(1,7):
    b = f"BILL_AMT{i}"
    p = f"PAY_AMT{i}"
    df_fe[f"PAY_RATIO{i}"] = df_fe[p] / (df_fe[b].abs() + 1)  # +1 to avoid division by zero

# --- Utilization proxy ---
df_fe["UTILIZATION6M_MEAN"] = (df_fe[bill_cols].clip(lower=0).mean(axis=1)) / (df_fe["LIMIT_BAL"] + 1)

# --- Aggregates over 6 months ---
df_fe["BILL_TOTAL_6M"] = df_fe[bill_cols].sum(axis=1)
df_fe["PAY_TOTAL_6M"]  = df_fe[pay_cols_amt].sum(axis=1)
df_fe["PAY_RATIO_6M_MEAN"] = df_fe[[f"PAY_RATIO{i}" for i in range(1,7)]].mean(axis=1)

# --- Delinquency features ---
pay_status_cols = ["PAY_0","PAY_2","PAY_3","PAY_4","PAY_5","PAY_6"]
df_fe["MAX_DELAY_6M"] = df_fe[pay_status_cols].max(axis=1)
df_fe["MEAN_DELAY_6M"] = df_fe[pay_status_cols].mean(axis=1)
df_fe["MONTHS_LATE_6M"] = (df_fe[pay_status_cols] > 0).sum(axis=1)
df_fe["RECENT_DELAY"] = df_fe["PAY_0"]  # closest month

# --- Trend features (simple slope) ---
def slope_6(values):
    x = np.arange(len(values))
    try:
        return np.polyfit(x, values, 1)[0]
    except Exception:
        return 0.0

df_fe["BILL_SLOPE_6M"] = df_fe[bill_cols].apply(lambda r: slope_6(r.values), axis=1)
df_fe["PAY_SLOPE_6M"]  = df_fe[pay_cols_amt].apply(lambda r: slope_6(r.values), axis=1)

print_df(df_fe, "df_fe (engineered)", n=5)

# Sanity check: engineered columns summary
section("Engineered features summary")
eng_cols = [c for c in df_fe.columns if "RATIO" in c or "UTIL" in c or "TOTAL" in c or "SLOPE" in c or "DELAY" in c or "MONTHS_LATE" in c]
print(df_fe[eng_cols].describe().T.to_string())


In [ ]:
# =============================================================================
# MODELING PREP (baseline) — optional but useful in an ML notebook
# =============================================================================
section("Modeling prep (baseline)")

from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import roc_auc_score, average_precision_score, classification_report, confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

target = "DEFAULT_NEXT_MONTH"

X = df_fe.drop(columns=[target], errors="ignore")
y = df_fe[target].astype(int)

# Define feature types
cat_features = ["SEX", "EDUCATION", "MARRIAGE"]
num_features = [c for c in X.columns if c not in cat_features and c != "ID"]

# Train/test split (stratified)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.20, random_state=RANDOM_STATE, stratify=y
)

preprocess = ColumnTransformer(
    transformers=[
        ("cat", OneHotEncoder(handle_unknown="ignore"), cat_features),
        ("num", StandardScaler(with_mean=False), num_features),
    ],
    remainder="drop"
)

# Baseline 1: Logistic Regression (interpretable)
lr = Pipeline(steps=[
    ("prep", preprocess),
    ("model", LogisticRegression(max_iter=2000, class_weight="balanced"))
])

# Baseline 2: Random Forest (non-linear)
rf = Pipeline(steps=[
    ("prep", preprocess),
    ("model", RandomForestClassifier(
        n_estimators=400,
        random_state=RANDOM_STATE,
        class_weight="balanced_subsample",
        n_jobs=-1
    ))
])

models = {
    "LogReg (balanced)": lr,
    "RandomForest": rf
}

for name, model in models.items():
    section(f"Training: {name}")
    model.fit(X_train, y_train)

    # predict probabilities
    proba = model.predict_proba(X_test)[:, 1]

    auc = roc_auc_score(y_test, proba)
    ap  = average_precision_score(y_test, proba)
    print(f"ROC AUC: {auc:.4f}")
    print(f"PR AUC : {ap:.4f}")

    # classification report at default threshold 0.5
    pred = (proba >= 0.5).astype(int)
    print("\nConfusion matrix:")
    print(confusion_matrix(y_test, pred))
    print("\nClassification report:")
    print(classification_report(y_test, pred, digits=4))



In [ ]:
# =============================================================================
# ADVANCED MODELS (optional) — Gradient Boosting / XGBoost / LightGBM / CatBoost
# =============================================================================
section("Advanced models (optional)")

# These are not always installed in every environment.
# If installed, they often outperform baselines on this dataset.

advanced_results = []

try:
    from xgboost import XGBClassifier

    xgb = Pipeline(steps=[
        ("prep", preprocess),
        ("model", XGBClassifier(
            n_estimators=600,
            learning_rate=0.05,
            max_depth=4,
            subsample=0.9,
            colsample_bytree=0.9,
            reg_lambda=1.0,
            eval_metric="logloss",
            random_state=RANDOM_STATE
        ))
    ])

    section("Training: XGBoost")
    xgb.fit(X_train, y_train)
    proba = xgb.predict_proba(X_test)[:, 1]
    advanced_results.append(("XGBoost", roc_auc_score(y_test, proba), average_precision_score(y_test, proba)))
except Exception as e:
    print("XGBoost not available or failed to import/train:", e)

try:
    import lightgbm as lgb

    lgbm = Pipeline(steps=[
        ("prep", preprocess),
        ("model", lgb.LGBMClassifier(
            n_estimators=1000,
            learning_rate=0.03,
            num_leaves=31,
            subsample=0.9,
            colsample_bytree=0.9,
            random_state=RANDOM_STATE
        ))
    ])

    section("Training: LightGBM")
    lgbm.fit(X_train, y_train)
    proba = lgbm.predict_proba(X_test)[:, 1]
    advanced_results.append(("LightGBM", roc_auc_score(y_test, proba), average_precision_score(y_test, proba)))
except Exception as e:
    print("LightGBM not available or failed to import/train:", e)

try:
    from catboost import CatBoostClassifier

    # CatBoost can handle categorical features natively, but we keep the same pipeline style for consistency.
    cat = Pipeline(steps=[
        ("prep", preprocess),
        ("model", CatBoostClassifier(
            iterations=1000,
            learning_rate=0.05,
            depth=6,
            loss_function="Logloss",
            verbose=False,
            random_seed=RANDOM_STATE
        ))
    ])

    section("Training: CatBoost")
    cat.fit(X_train, y_train)
    proba = cat.predict_proba(X_test)[:, 1]
    advanced_results.append(("CatBoost", roc_auc_score(y_test, proba), average_precision_score(y_test, proba)))
except Exception as e:
    print("CatBoost not available or failed to import/train:", e)

if advanced_results:
    section("Advanced models summary")
    for name, auc, ap in sorted(advanced_results, key=lambda x: x[1], reverse=True):
        print(f"{name:10s} | ROC AUC={auc:.4f} | PR AUC={ap:.4f}")
else:
    print("No advanced model results available (libraries may not be installed).")
